# Eff. matching to GEN particle


   * [Efficiency vs $\eta$](#Effinciency-vs-$\eta$)
   * [Efficiency vs $p_{T}$](#Efficiency-vs-$p_{T}$)
   * [Turn-on curves](#Turn-on-curves)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [16]:
# %load drawingTools.py
import ROOT
import math
import uuid
import pandas as pd

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 12)
stuff = []
f_idx = 0

ROOT.gStyle.SetOptTitle(False)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.13)

# ROOT.gStyle.SetCanvasBorderMode(0)
# ROOT.gStyle.SetCanvasColor(0)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def newCanvas(name=None, title=None, height=600, width=800, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title, width, height
    canvas = ROOT.TCanvas(name, title, width, height)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def draw(histograms,
         labels,
         options='',
         norm=False,
         logy=False,
         min_y=None,
         max_y=None,
         text=None,
         y_axis_label=None,
         x_axis_label=None,
         v_lines=None,
         h_lines=None,
         do_stats=False,
         do_profile=False,
         do_ratio=False):
    global colors
    global stuff
    global p_idx

    # 0 - determine kind and # of histos
    n_histos = len(histograms)
    if n_histos == 0:
        print '[draw]**ERROR: empy histo list'
        return

    histo_class = histograms[0].ClassName()

    # 1 - check argument consisntency
    if 'TH2' not in histo_class and do_profile:
        print '[draw]**ERROR: do_profile set for histo of class: {}'.format(histo_class)
        return

    # TH1 histograms are overlayed
    # TH2 histograms are drawn side by side in the same canvas
    overlay = True
    if n_histos > 1:
        if 'TH2' in histo_class or 'TH3' in histo_class:
            overlay = False

    # 2 - determine canvas properties
    xdiv = 0
    ydiv = 0
    c_width = 800
    c_height = 600
    # do_ratio = False

    if do_ratio:
        c_height = 800

    if not overlay:
        xdiv = 2
        c_width = 1000.
        ydiv = math.ceil(float(n_histos)/2)
        c_height = 500*ydiv
    canvas = newCanvas(name=None,
                       title=None,
                       height=int(c_height),
                       width=int(c_width),
                       xdiv=int(xdiv),
                       ydiv=int(ydiv))
    if overlay:
        canvas.SetRightMargin(0.30)

    canvas.cd()
    leg = getLegend()

    drawn_histos = []
    # drawn_histos = histograms
    for hidx, hist in enumerate(histograms):
        opt = options
        if 'TGraph' not in histo_class:
            hist.SetStats(do_stats)
        else:
            opt = 'P'+options
            hist.SetMarkerSize(3)
            hist.SetMarkerStyle(7)
        if overlay:
            hist.SetLineColor(colors[hidx])
            if 'TGraph' in histo_class:
                hist.SetMarkerColor(colors[hidx])
                # hist.SetMarkerSize(3)
                # hist.SetMarkerStyle(7)

            if hidx:
                opt = 'same,'+opt
            else:
                if 'TGraph' in histo_class:
                    opt = opt+'A'
        else:
            canvas.cd(hidx+1)
        # print hidx, opt
        d_hist = hist
        if norm:
            d_hist = hist.DrawNormalized(opt, 1.)
        else:
            hist.Draw(opt)

        if do_profile:
            profname = d_hist.GetName()+'_prof_'+str(p_idx)
            p_idx += 1
            firstbin = 1
            lastbin = -1
            prof = d_hist.ProfileX(profname, firstbin, lastbin, 's')
            prof.SetMarkerColor(2)
            prof.SetLineColor(2)
            prof.Draw('same')

        if overlay:
            if 'TGraph' not in histo_class:
                leg.AddEntry(hist, labels[hidx], 'l')
            else:
                leg.AddEntry(hist, labels[hidx], 'P')
        else:
            if text:
                newtext = '{}: {}'.format(labels[hidx], text)
                rtext = getText(newtext, 0.15, 0.85)
                rtext.Draw('same')
        drawn_histos.append(d_hist)

    if do_ratio:
        h_ratio = ROOT.TRatioPlot(drawn_histos[0], drawn_histos[1])
        stuff.append(h_ratio)
        h_ratio.Draw()

        pad = canvas.cd(1)
        h_ratio.GetUpperPad().SetBottomMargin(0)
        h_ratio.GetUpperPad().SetRightMargin(0.3)
        if logy:
            h_ratio.GetUpperPad().SetLogy()
        pad.Update()
        pad = canvas.cd(2)
        h_ratio.GetLowerPad().SetTopMargin(0)
        h_ratio.GetLowerPad().SetRightMargin(0.3)
        pad.Update()
        # hratio.GetLowerRefGraph().GetYaxis().SetRangeUser(-6, 6)

    canvas.Update()

    # we now set the axis properties
    max_value = max_y
    min_value = min_y

    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in drawn_histos])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in drawn_histos])*1.2
    # print min_value, max_value

    for hist in drawn_histos:
        hist.GetYaxis().SetRangeUser(min_value, max_value)
        if y_axis_label:
            hist.GetYaxis().SetTitle(y_axis_label)
        if x_axis_label:
            hist.GetXaxis().SetTitle(x_axis_label)

    canvas.Draw()

    # we draw additional stuff if needed
    if overlay:
        canvas.cd()
        leg.Draw()
        if text:
            rtext = getText(text, 0.15, 0.85)
            rtext.Draw("same")

    pad_range = range(0, 1)
    if not overlay:
        pad_range = range(1, len(histograms)+1)

    for pad_id in pad_range:
        canvas.cd(pad_id)
        if v_lines:
            for v_line_x in v_lines:
                aline = ROOT.TLine(v_line_x, ROOT.gPad.GetUymin(), v_line_x,  ROOT.gPad.GetUymax())
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if h_lines:
            for h_line_y in h_lines:
                aline = ROOT.TLine(ROOT.gPad.GetUxmin(), h_line_y, ROOT.gPad.GetUxmax(),  h_line_y)
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if logy:
            if not do_ratio:
                ROOT.gPad.SetLogy()

        ROOT.gPad.Update()
    canvas.Update()
    canvas.Draw()

    return canvas


files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = ROOT.TFile(self.histo_filename, 'r')
        self.type = type

    def __repr__(self):
        return '<{} {}, {}>'.format(self.__class__.__name__, self.histo_filename, self.type)

# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self, debug=False):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            if debug:
                print '-- HProxy:Get: {}'.format(name)
            self.instance = self.classtype(name, self.root_file, debug)
        return self.instance


class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        # self.tp_selections = tp_selections
        # self.gen_selections = gen_selections
        self.pus = []
        self.labels_dict = {}

        for sample in samples:
            self.pus.append(sample.label)
            self.labels_dict[sample.type] = sample.type

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        if gen_sels is None:
            gen_sels = [None]

        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    for gen_sel in gen_sels:
                        print sample, tp, tp_sel, gen_sel
                        self.data = self.data.append({'sample': sample.type,
                                                      'pu': sample.label,
                                                      'tp': tp,
                                                      'tp_sel': tp_sel,
                                                      'gen_sel': gen_sel,
                                                      'classtype': classtype,
                                                      'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)},
                                                     ignore_index=True)

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None,
                  debug=False):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += ' & (gen_sel == @gen_sel)'
        else:
            query += ' & (gen_sel.isnull())'
        if sample is not None:
            query += '& (sample == @sample)'

        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
        if debug:
            print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get(debug) for his in histo_df['histo'].values]
        return histo, labels, text


In [3]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections
import python.collections as collections

version = 'v81'

version_V9 = 'v91'


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0_tracks', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200_tracks', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0_tracks', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200_tracks', 'PU200', version, 'V8')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0_tracks', 'PU0', version_V9, 'ele-V9'),
    Sample('ele_flat2to100_PU200_tracks', 'PU200', 'v94', 'ele-V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0_tracks', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200_tracks', 'PU200', version, 'photon')
    ]

samples_photons_V9 = [
    Sample('photon_flat8to150_PU0_tracks', 'PU0', version_V9, 'photon-V9'),
    Sample('photon_flat8to150_PU200_tracks', 'PU200', version_V9, 'photon-V9')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0_tracks', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200_tracks', 'PU200', version, 'mb')
    ]


event_manager = collections.EventManager()

all_tpsets = {}
for collection in event_manager.collections:
    all_tpsets[collection.name] = collection.label

    
tpsets = {}
tpset_selections = {}
gen_selections = {}

samples = []

# tpset_selections.update(get_label_dict(tp_rate_selections))
tpset_selections.update(get_label_dict(selections.tp_match_selections))

gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})


gen_part_selections: 8
[EventManager] registering collection: MC
[EventManager] registering collection: GEN
[EventManager] registering collection: TC
[EventManager] registering collection: TCTrue
[EventManager] registering collection: DEF2D
[EventManager] registering collection: DEF2DTrue
[EventManager] registering collection: HMvDRTrue
[EventManager] registering collection: DEF
[EventManager] registering collection: DEFNC
[EventManager] registering collection: HMvDR
[EventManager] registering collection: HMvDRRebin
[EventManager] registering collection: HMvDRsTC
[EventManager] registering collection: HMvDRNC0
[EventManager] registering collection: HMvDRNC1
[EventManager] registering collection: DEFMerged
[EventManager] registering collection: DEFCalib
[EventManager] registering collection: HMvDRMerged
[EventManager] registering collection: HMvDRfixed
[EventManager] registering collection: HMvDRcylind10
[EventManager] registering collection: HMvDRcylind5
[EventManager] registering coll

Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_tracks_v81.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_tracks_v81.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_tracks_v81.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_tracks_v81.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_tracks_v81.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_tracks_v81.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_tracks_v91.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_tracks_v91.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_tracks_v81.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v81.root does not exist
Error in <TFile::T

In [4]:
# %load settings.py
# from notebooks.samples import get_label_dict
# import selections
import pprint

# === samples =====================================================
samples = []

# samples = samples_ele_V8
samples += samples_ele_V9
# samples += samples_photon_V9


sample = 'ele-V9'

do_rate = False

# === TP ==========================================================
tps = ['EG', 'TkEG', 'TkEle', 'TkIsoEle', 'L1Trk', 'L1TrkEmu']
# tps = ['DEF', 'HMvDR']

# === TP selections ================================================
tp_select = {}
tp_select['DEF'] = ['Em', 'all']
tp_select['DEFNC'] = tp_select['DEF']
tp_select['HMvDR'] = tp_select['DEF']
tp_select['HMvDRNC0'] = tp_select['DEF']
tp_select['HMvDRNC1'] = tp_select['DEF']


tp_select['EG'] = ['EGq2', 'EGq3']

tp_select['TkEG'] = get_label_dict(selections.tkeg_qual_selections).keys()
tp_select['TkEle'] = ['EGq2', 'EGq3']

tp_select['TkEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['TkIsoEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['L1Trk'] = ['all', 'Pt2', 'Pt10']
tp_select['L1TrkEmu'] = ['all', 'Pt2', 'Pt10']

# ==== GEN selections ===============================================

gen_select = ['GENEtaBC']
# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]

# tp_select['TkEG'] = ['EGq2EtaBC', 'EGq3EtaBC', 'EGq2EtaBCM2', 'EGq3EtaBCM2', 'EGq2EtaBCM2s', 'EGq3EtaBCM2s','EGq2EtaBCM3', 'EGq3EtaBCM3', 'EGq2EtaBCM3s', 'EGq3EtaBCM3s','EGq2EtaBCM4', 'EGq3EtaBCM4']


# ==== adapt the plot retrieval
tpsets = {}
tpset_selections = {}
gen_selections = {}

for tp in tps:
    tpsets[tp] = all_tpsets[tp]

    
    
if 'DEF' in tps or 'HMvDR' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tp_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tp_match_selections))
if 'EG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.eg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.eg_pt_selections))
if 'TkEG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkeg_pt_selections))
if 'TkEle' in tps or 'TkIsoEle' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkisoeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkisoeg_pt_selections))
if 'L1Trk' in tps:
    tpset_selections.update(get_label_dict(selections.tracks_selections))


gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})


pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
print gen_select


--- TPs: 
['EG', 'TkEG', 'TkEle', 'TkIsoEle', 'L1Trk', 'L1TrkEmu']
--- TP selections:
{   'DEF': ['Em', 'all'],
    'DEFNC': ['Em', 'all'],
    'EG': ['EGq2', 'EGq3'],
    'HMvDR': ['Em', 'all'],
    'HMvDRNC0': ['Em', 'all'],
    'HMvDRNC1': ['Em', 'all'],
    'L1Trk': ['all', 'Pt2', 'Pt10'],
    'L1TrkEmu': ['all', 'Pt2', 'Pt10'],
    'TkEG': [   'EGq2M1P2',
                'EGq2M1P2S',
                'EGq3M1P10',
                'EGq2M1P5',
                'EGq2M1P10',
                'EGq3M1',
                'EGq3M1P5S',
                'EGq3',
                'EGq2',
                'EGq3M1P2S',
                'EGq2M1P5S',
                'EGq2M1P10S',
                'EGq3M1P2',
                'EGq2M1',
                'EGq3M1P5',
                'EGq3M1P10S'],
    'TkEle': [   'EGq2',
                 'EGq3',
                 'EGq2Iso0p2',
                 'EGq3Iso0p2',
                 'EGq2Iso0p3',
                 'EGq3Iso0p3'],
    'TkIsoEle': [   'EGq2',
               

In [5]:
from python.plotters import eg_genmatched_plotters, track_genmatched_plotters

histo_primitives = pd.DataFrame() 

for plotter in eg_genmatched_plotters:
    histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
for plotter in track_genmatched_plotters:
    histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)

    
print histo_primitives.data.unique()

['EG' 'TkEG' 'TkEle' 'TkIsoEle' 'L1Trk' 'L1TrkEmu']


In [6]:
for tp in tps:
    print 'TP: {}'.format(tp)
    print histo_primitives[histo_primitives.data == tp].data_sel.unique()
    print histo_primitives[histo_primitives.data == tp].gen_sel.unique()

TP: EG
['EGq2' 'EGq2Pt10' 'EGq2Pt20' 'EGq2Pt30' 'EGq3' 'EGq3Pt10' 'EGq3Pt20'
 'EGq3Pt30']
['GEN' 'GENPt10' 'GENPt20' 'GENPt30' 'GENPt40' 'GENEtaD' 'GENEtaBC'
 'GENEtaBCD']
TP: TkEG
['EGq2' 'EGq2Pt10' 'EGq2Pt20' 'EGq2Pt30' 'EGq2M1' 'EGq2M1Pt10' 'EGq2M1Pt20'
 'EGq2M1Pt30' 'EGq2M1P2' 'EGq2M1P2Pt10' 'EGq2M1P2Pt20' 'EGq2M1P2Pt30'
 'EGq2M1P5' 'EGq2M1P5Pt10' 'EGq2M1P5Pt20' 'EGq2M1P5Pt30' 'EGq2M1P10'
 'EGq2M1P10Pt10' 'EGq2M1P10Pt20' 'EGq2M1P10Pt30' 'EGq2M1P2S'
 'EGq2M1P2SPt10' 'EGq2M1P2SPt20' 'EGq2M1P2SPt30' 'EGq2M1P5S'
 'EGq2M1P5SPt10' 'EGq2M1P5SPt20' 'EGq2M1P5SPt30' 'EGq2M1P10S'
 'EGq2M1P10SPt10' 'EGq2M1P10SPt20' 'EGq2M1P10SPt30' 'EGq3' 'EGq3Pt10'
 'EGq3Pt20' 'EGq3Pt30' 'EGq3M1' 'EGq3M1Pt10' 'EGq3M1Pt20' 'EGq3M1Pt30'
 'EGq3M1P2' 'EGq3M1P2Pt10' 'EGq3M1P2Pt20' 'EGq3M1P2Pt30' 'EGq3M1P5'
 'EGq3M1P5Pt10' 'EGq3M1P5Pt20' 'EGq3M1P5Pt30' 'EGq3M1P10' 'EGq3M1P10Pt10'
 'EGq3M1P10Pt20' 'EGq3M1P10Pt30' 'EGq3M1P2S' 'EGq3M1P2SPt10'
 'EGq3M1P2SPt20' 'EGq3M1P2SPt30' 'EGq3M1P5S' 'EGq3M1P5SPt10'
 'EGq3M1P5SPt20

In [7]:
tpset_selections

{'EGq2': 'q2',
 'EGq2Iso0p2': 'q2, Iso0p2',
 'EGq2Iso0p2Pt10': 'q2, Iso0p2, p_{T}^{L1}>=10GeV',
 'EGq2Iso0p2Pt20': 'q2, Iso0p2, p_{T}^{L1}>=20GeV',
 'EGq2Iso0p2Pt30': 'q2, Iso0p2, p_{T}^{L1}>=30GeV',
 'EGq2Iso0p3': 'q2, Iso0p3',
 'EGq2Iso0p3Pt10': 'q2, Iso0p3, p_{T}^{L1}>=10GeV',
 'EGq2Iso0p3Pt20': 'q2, Iso0p3, p_{T}^{L1}>=20GeV',
 'EGq2Iso0p3Pt30': 'q2, Iso0p3, p_{T}^{L1}>=30GeV',
 'EGq2M1': 'q2, |#Delta#phi| <0.08 & #DeltaR < 0.07',
 'EGq2M1P10': 'q2, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV',
 'EGq2M1P10Pt10': 'q2, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV, p_{T}^{L1}>=10GeV',
 'EGq2M1P10Pt20': 'q2, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV, p_{T}^{L1}>=20GeV',
 'EGq2M1P10Pt30': 'q2, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV, p_{T}^{L1}>=30GeV',
 'EGq2M1P10S': 'q2, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV',
 'EGq2M1P10SPt10': 'q2, |#Delta#phi| <0.08 & #DeltaR < 0.07 & p_{T}^{trk} > 10GeV, p_{T}^{L1}>=1

In [8]:
gen_selections

{'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': ''}

In [9]:
%%time
# for key in samples_ele[0].histo_file.GetListOfKeys():
#     #pass
#     key.GetName()
# print len(samples_ele[0].histo_file.GetListOfKeys())
# l = [histo for histo in samples_ele[0].histo_file.GetListOfKeys()]
l = [(histo.ReadObj(), histo.GetName()) for histo in samples_ele[0].histo_file.GetListOfKeys() if 'DEF_all'+'_' in histo.GetName()]

CPU times: user 5.1 ms, sys: 550 µs, total: 5.65 ms
Wall time: 5.7 ms


In [10]:
print dir(samples[0])
print samples[0].histo_filename

['__doc__', '__init__', '__module__', 'histo_file', 'histo_filename', 'label', 'name', 'type']
../plots1/histos_ele_flat2to100_PU0_tracks_v91.root


In [11]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)

for tp in tps:
    hplot.cache_histo(classtype=histos.HistoSetEff, 
                      samples=samples,
                      pus=[],
                      tps=[tp],
                      tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
                      gen_sels=gen_selections)



<drawingTools.Sample instance at 0x11682f488>
<drawingTools.Sample instance at 0x11682f488> EG EGq2 GENPt40
<drawingTools.Sample instance at 0x11682f488> EG EGq2 GENEtaD
<drawingTools.Sample instance at 0x11682f488> EG EGq2 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> EG EGq2 GENPt30
<drawingTools.Sample instance at 0x11682f488> EG EGq2 GENPt20
<drawingTools.Sample instance at 0x11682f488> EG EGq2 GENEtaBCD
<drawingTools.Sample instance at 0x11682f488> EG EGq2 GENPt10
<drawingTools.Sample instance at 0x11682f488> EG EGq2 nomatch
<drawingTools.Sample instance at 0x11682f488> EG EGq2 GEN
<drawingTools.Sample instance at 0x11682f488> EG EGq2Pt10 GENPt40
<drawingTools.Sample instance at 0x11682f488> EG EGq2Pt10 GENEtaD
<drawingTools.Sample instance at 0x11682f488> EG EGq2Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> EG EGq2Pt10 GENPt30
<drawingTools.Sample instance at 0x11682f488> EG EGq2Pt10 GENPt20
<drawingTools.Sample instance at 0x11682f488> EG EGq2Pt10 GENEtaBC

<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt10 GENPt40
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt10 GENEtaD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt10 GENPt30
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt10 GENPt20
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt10 GENPt10
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt10 nomatch
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt10 GEN
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt20 GENPt40
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt20 GENEtaD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt20 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt20 GENPt30
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2Pt20 GENPt20
<drawingTools.Sample instance at 0x11682f488> Tk

<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P5Pt30 GEN
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10 GENPt40
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10 GENEtaD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10 GENPt30
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10 GENPt20
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10 GENEtaBCD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10 GENPt10
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10 nomatch
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10 GEN
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10Pt10 GENPt40
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10Pt10 GENEtaD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10Pt10 GENPt30
<drawingTools.Sampl

<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt20 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt20 GENPt30
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt20 GENPt20
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt20 GENEtaBCD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt20 GENPt10
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt20 nomatch
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt20 GEN
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt30 GENPt40
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt30 GENEtaD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt30 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt30 GENPt30
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt30 GENPt20
<drawingTools.Sample instance at 0x11682f488> TkEG EGq2M1P10SPt30 GENEtaBCD
<drawingTools.Sample instance at 0x1

<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt10 GENPt20
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt10 GENPt10
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt10 nomatch
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt10 GEN
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt20 GENPt40
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt20 GENEtaD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt20 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt20 GENPt30
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt20 GENPt20
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt20 GENEtaBCD
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt20 GENPt10
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt20 nomatch
<drawingTools.Sample instance at 0x11682f488> TkEG EGq3M1P5Pt20

...

<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3 GENPt40
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3 GENEtaD
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3 GENPt30
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3 GENPt20
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3 GENEtaBCD
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3 GENPt10
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3 nomatch
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3 GEN
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3Pt10 GENPt40
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3Pt10 GENEtaD
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> TkIsoEle EGq2Iso0p3Pt10 GENPt30
<drawingTools.Sample i

<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2 GENEtaBC
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2 GENPt30
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2 GENPt20
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2 GENEtaBCD
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2 GENPt10
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2 nomatch
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2 GEN
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2Pt10 GENPt40
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2Pt10 GENEtaD
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2Pt10 GENPt30
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2Pt10 GENPt20
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq2Pt10 GENPt10
<drawingTools.Samp

<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3 GEN
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt10 GENPt40
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt10 GENEtaD
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt10 GENPt30
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt10 GENPt20
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt10 GENPt10
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt10 nomatch
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt10 GEN
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt20 GENPt40
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt20 GENEtaD
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt20 GENEtaBC
<drawingTools.Sample instance at 0x11682f5a8> TkIsoEle EGq3Pt20 GENPt30


<drawingTools.Sample instance at 0x11682f488> L1Trk Pt10 GENEtaD
<drawingTools.Sample instance at 0x11682f488> L1Trk Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> L1Trk Pt10 GENPt30
<drawingTools.Sample instance at 0x11682f488> L1Trk Pt10 GENPt20
<drawingTools.Sample instance at 0x11682f488> L1Trk Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x11682f488> L1Trk Pt10 GENPt10
<drawingTools.Sample instance at 0x11682f488> L1Trk Pt10 nomatch
<drawingTools.Sample instance at 0x11682f488> L1Trk Pt10 GEN
<drawingTools.Sample instance at 0x11682f488> L1Trk St4 GENPt40
<drawingTools.Sample instance at 0x11682f488> L1Trk St4 GENEtaD
<drawingTools.Sample instance at 0x11682f488> L1Trk St4 GENEtaBC
<drawingTools.Sample instance at 0x11682f488> L1Trk St4 GENPt30
<drawingTools.Sample instance at 0x11682f488> L1Trk St4 GENPt20
<drawingTools.Sample instance at 0x11682f488> L1Trk St4 GENEtaBCD
<drawingTools.Sample instance at 0x11682f488> L1Trk St4 GENPt10
<drawingTools.Sample instance 

<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt2 GENEtaBCD
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt2 GENPt10
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt2 nomatch
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt2 GEN
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt10 GENPt40
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt10 GENEtaD
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt10 GENPt30
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt10 GENPt20
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt10 GENPt10
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt10 nomatch
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu Pt10 GEN
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu St4 GENPt40
<drawingTools.Sample instance at 0x11682f5a8> L1TrkEmu S

In [12]:
print dir(samples[0])
samples[0].histo_file.ls()

['__doc__', '__init__', '__module__', 'histo_file', 'histo_filename', 'label', 'name', 'type']
TFile**		../plots1/histos_ele_flat2to100_PU0_tracks_v91.root	
 TFile*		../plots1/histos_ele_flat2to100_PU0_tracks_v91.root	
  KEY: TDirectoryFile	TrackHistos;1	TrackHistos
  KEY: TDirectoryFile	TrackResoHistos;1	TrackResoHistos
  KEY: TDirectoryFile	GenParticleHistos;1	GenParticleHistos
  KEY: TDirectoryFile	TkEGHistos;1	TkEGHistos
  KEY: TDirectoryFile	EGHistos;1	EGHistos
  KEY: TDirectoryFile	EGResoHistos;1	EGResoHistos


In [13]:
%%time

for smp in hplot.data['sample'].unique():
    print 'Sample: {}'.format(smp)
    for pu in hplot.data[(hplot.data['sample'] == smp)].pu.unique():
        for tp in ['EG', 'TkEle', 'TkIsoEle', 'TkEG']:
            print 'PU: {}, TP: {}'.format(pu, tp)
            for tp_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp)].tp_sel.unique():
                if 'Pt' not in tp_sel:
                    continue
#                 tp_sel_den = 'all'
                # print tp_sel
                tp_sel_den = tp_sel.split('Pt')[0]
                if tp_sel_den == '':
                    tp_sel_den = 'all'
                print '  {} den -> {}'.format(tp_sel, tp_sel_den) 
                for gen_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp) & (hplot.data.tp_sel == tp_sel)].gen_sel.unique():
                    if gen_sel == 'nomatch' or 'Pt' in gen_sel:
                        continue
                    print gen_sel
                    hsetden = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel_den, gen_sel)
                    hset = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel, gen_sel)
                    hset[0][0].computeTurnOn(hsetden[0][0].h_num)


Sample: ele-V9
PU: PU0, TP: EG
  EGq2Pt10 den -> EGq2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Pt20 den -> EGq2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Pt30 den -> EGq2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq3Pt10 den -> EGq3
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq3Pt20 den -> EGq3
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq3Pt30 den -> EGq3
GENEtaD
GENEtaBC
GENEtaBCD
GEN
PU: PU0, TP: TkEle
  EGq2Pt10 den -> EGq2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Pt20 den -> EGq2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Pt30 den -> EGq2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Iso0p2Pt10 den -> EGq2Iso0p2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Iso0p2Pt20 den -> EGq2Iso0p2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Iso0p2Pt30 den -> EGq2Iso0p2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Iso0p3Pt10 den -> EGq2Iso0p3
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Iso0p3Pt20 den -> EGq2Iso0p3
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Iso0p3Pt30 den -> EGq2Iso0p3
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq3Pt10 den -> EGq3
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq3Pt20 den

GENEtaBC
GENEtaBCD
GEN
PU: PU200, TP: TkEG
  EGq2Pt10 den -> EGq2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Pt20 den -> EGq2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2Pt30 den -> EGq2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1Pt10 den -> EGq2M1
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1Pt20 den -> EGq2M1
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1Pt30 den -> EGq2M1
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1P2Pt10 den -> EGq2M1P2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1P2Pt20 den -> EGq2M1P2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1P2Pt30 den -> EGq2M1P2
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1P5Pt10 den -> EGq2M1P5
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1P5Pt20 den -> EGq2M1P5
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1P5Pt30 den -> EGq2M1P5
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1P10Pt10 den -> EGq2M1P10
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1P10Pt20 den -> EGq2M1P10
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1P10Pt30 den -> EGq2M1P10
GENEtaD
GENEtaBC
GENEtaBCD
GEN
  EGq2M1P2SPt10 den -> EGq2M1P2S
GENEtaD
GENEtaBC
GENEtaB

In [ ]:
hplot.data[:10]

In [ ]:
hplot.data[hplot.data.tp == 'EG'].tp_sel.unique()

In [ ]:
sample

In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0'], 'EG', 'EGq2', ['GENEtaBC'], debug=False)


## Effinciency vs $\eta$

In [ ]:
dir(hplot)


In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel, debug=False)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
#             print [his.h_num.h_eta.GetName() for his in hsets]

In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
     

In [ ]:
# for tp in tps:
#     for tp_sel in tp_select[tp]:
#         for gen_sel in ['GENPt30']:
for pu in ['PU0', 'PU200']:
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'all', ['GENPt30'])
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
for pu in ['PU0', 'PU200']:

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'Pt2', ['GENPt30'])
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.8, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
for pu in ['PU0', 'PU200']:
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'Pt10', ['GENPt30'])
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.8, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
    

In [ ]:
for pu in ['PU0', 'PU200']:
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'St4Pt10', ['GENPt30'])
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.8, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
    

In [ ]:
for pu in ['PU0', 'PU200']:
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'St4Pt2', ['GENPt30'])
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.8, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0'], ['L1Trk', 'L1TrkEmu'], 'all', ['GENPt30'])
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
# for tp in tps:
for pu in ['PU0', 'PU200']:
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle', 'TkIsoEle'], ['EGq2Iso0p2'], 'GENPt20')
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
for pu in ['PU0', 'PU200']:
    for tp in ['TkEG']:
#     for tp_sel in tp_select[tp]:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, ['EGq2' ,'EGq2M2','EGq2M2s','EGq2M3','EGq2M3s', 'EGq2M4'], gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
     

In [ ]:
tp_select['TkEG']

In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GENPt20']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'V8', ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
for pu in ['PU0', 'PU200']:
    for tp in ['EG', 'TkEle']:
        for tp_sel in tp_select[tp]:
            for gen_sel in ['GENPt20']:
                print tp, tp_sel
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['V8', 'V9'], pu, tp, tp_sel, gen_sel)
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
# for tp in tps:
#     for tp_sel in tp_select[tp]:
#         for gen_sel in ['GENPt20']:

for pu in ['PU0', 'PU200']:

    hsets_new = []
    labels_new = []
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq2'], 'GENPt20')
    hsets_new.append(hsets[0])
    labels_new.append("EG standalone")
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['Pt2'], 'GENPt20')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>2GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['Pt10'], 'GENPt20')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq2'], 'GENPt20')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")

    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


## Efficiency vs $p_{T}$

In [ ]:
line_y1p0 = ROOT.TLine(0.,1., 110., 1.)
line_y0p95 = ROOT.TLine(0.,0.95, 110., 0.95)
line_y0p95.SetLineStyle(2)

In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
    for gen_sel in gen_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'V8', pu, ['TkEle', 'TkIsoEle'], 'EGq2Iso0p2', gen_sel)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
 

In [ ]:
for pu in ['PU0']:
    for gen_sel in gen_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'all', gen_sel)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
 

In [ ]:
for pu in ['PU0', 'PU200']:
    for gen_sel in gen_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'Pt2', gen_sel)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
 

In [ ]:
for pu in ['PU0', 'PU200']:
    for gen_sel in gen_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'Pt10', gen_sel)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
 

In [18]:
sample = 'ele-V9'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("EG standalone")
    
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3Pt20'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("EG Pt>20GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>2GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['Pt10'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEle")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P10'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEG")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P5'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP5")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P2'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP2")

    text = '{}, HWQual==3, 1.52<|{}|<=2.4'.format(pu, '#eta^{GEN}')
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_pt.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [ ]:
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq3M1P10', 'EGq3M1P5', 'EGq3M1P2'], 'GENEtaBC')
    hsets_new.append(hsets[2])
    labels_new.append('p_{T}^{track}>10GeV')
    hsets_new.append(hsets[1])
    labels_new.append('p_{T}^{track}>5GeV')

    hsets_new.append(hsets[0])
    labels_new.append('p_{T}^{track}>2GeV')

    #     labels_new.append("TkEG")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P5'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP5")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P2'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP2")

    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_pt.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.7, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [ ]:
sample = 'V8'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P10', 'EGq2M1P5S', 'EGq2M1P2S'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P5'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP5")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P2'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP2")

    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0.7, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


## Efficiency vs fBrem

In [ ]:
sample = 'ele-V9'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3'], 'GENEtaBC')
# #     hsets[0].SetMarkerStyle(7)
#     hsets_new.append(hsets[0])
#     labels_new.append("EG standalone")
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3Pt20'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("EG Pt>20GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['St4Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk #stubs > 3")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['St4Pt10'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk EMU")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt10'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("L1Trk p_{T}>10GeV")

    
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEle")


    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_fBrem.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [26]:
sample = 'ele-V9'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3'], 'GENEtaBC')
# #     hsets[0].SetMarkerStyle(7)
#     hsets_new.append(hsets[0])
#     labels_new.append("EG standalone")
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3Pt20'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("EG Pt>20GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>2GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt10'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq3M1P10'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")


    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_fBrem.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


## Turn-on curves

In [ ]:
for tp in tps:
#     if tp in ['TkEle', 'TkIsoEle', 'L1Trk']:
#         continue
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            tp_sel+='Pt30'
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel, debug=False)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
for tp in tps:
    if tp in ['TkEG', 'TkIsoEle', 'L1Trk']:
        continue
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            tp_sel+='Pt20'
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel, debug=False)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
for tp in tps:
    if tp in ['TkEG', 'TkIsoEle', 'L1Trk']:
        continue
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            tp_sel+='Pt10'
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel, debug=True)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:

for pu in ['PU0']:
    for gen_sel in gen_select:
        tp_sel+='Pt20'
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq2Pt20', 'EGq3Pt20'], gen_sel, debug=True)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
for tp in tps:
    for tp_sel in ['Pt20', 'EmPt20', 'Emv1Pt20', 'Pt30', 'EmPt30', 'Emv1Pt30']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            drawSame([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
    for tp in tps:
        for tp_sel in ['Pt20', 'EmPt20', 'Emv1Pt20', 'Pt30', 'EmPt30', 'Emv1Pt30']:
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, ['GENEtaABC', 'GENEtaBC'])
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                drawSame([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
        for tp_sel in ['EmPt20', 'Emv1Pt20', 'EmPt30', 'Emv1Pt30']:
            for gen_sel in gen_select:
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, ['DEF', 'DEFCalib'], tp_sel, gen_sel)
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                drawSame([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
        for tp_sel in ['EmPt20', 'Emv1Pt20', 'EmPt30', 'Emv1Pt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, ['DEF', 'DEFCalib'], tp_sel, ['GENEtaABC', 'GENEtaBC'])
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            drawSame([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

## Scaling

In [19]:
def f_yc(x, par):
    return (ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], 0) - ROOT.TMath.Exp(-par[0]*(x[0]-par[1])+par[0]*par[0]*par[2]*par[2]/2)*ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], par[0]*par[0]*par[2]*par[2])) * (par[3] - par[4]) + par[4]

tf_yc = ROOT.TF1('tf_yc', f_yc, 0, 100, 5)
tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
tf_yc.SetParNames("#lambda", "#mu", "#sigma", "Plateau", "Baseline");
tf_yc.SetLineColor(6)
tf_yc.FixParameter(4,0)

tf_yc.SetParLimits(3, 0.8, 1)
tf_yc.SetParLimits(0, 0, 1)

c = newCanvas()
tf_yc.Draw()
c.Draw()


In [20]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

import array

class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo



for pu in ['PU0', 'PU200']:
    for gen_sel in ['GENEtaBC']:
        for tp in ['EG', 'TkEle']:
            pt_points = [10, 20, 30]
            pt_95 = []
            for pt in pt_points:
                tp_sel = 'EGq3Pt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel, gen_sel)
                    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    #                 draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]

                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                pt_95.append(pt095)

                #                 points.append((tp, pu, pt, pt095))
                draw(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

            graph = ROOT.TGraph(len(pt_points), array.array('d', pt_points), array.array('d', pt_95))
            graph.SetMarkerStyle(7)
            graph.SetMarkerColor(2)
            stuff.append(graph)

            hplot.data = hplot.data.append({'sample': sample,
                                            'pu': pu,
                                            'tp': tp,
                                            'tp_sel': 'EGq3',
                                            'gen_sel': gen_sel,
                                            'classtype': 'pippo',
                                            'histo': HWrapper(graph),},
                                            ignore_index=True)

                
                

------- PU0, EG, q3, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, EG, q3, p_{T}^{L1}>=20GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, EG, q3, p_{T}^{L1}>=30GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, TkEle, q3, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, TkEle, q3, p_{T}^{L1}>=20GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, TkEle, q3, p_{T}^{L1}>=30GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, TkEG, q3, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, TkEG, q3, p_{T}^{L1}>=20GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, TkEG, q3, p_{T}^{L1}>=30GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU200, EG, q3, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU200, EG, q3, p_{T}^{L1}>=20GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU200, EG, q3, p_{T}^{L1}>=30GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU200, TkEle, q3, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU200, TkEle, q3, p_{T}^{L1}>=

In [22]:
hplot.data[hplot.data.classtype == 'pippo']


,sample,pu,tp,tp_sel,gen_sel,classtype,histo
2376,ele-V9,PU0,EG,EGq3,GEN,pippo,<__main__.HWrapper object at 0x11742c590>
2377,ele-V9,PU0,TkEle,EGq3,GEN,pippo,<__main__.HWrapper object at 0x116840f50>
2378,ele-V9,PU0,TkEG,EGq3,GEN,pippo,<__main__.HWrapper object at 0x116a0c050>
2379,ele-V9,PU200,EG,EGq3,GEN,pippo,<__main__.HWrapper object at 0x116a1fd90>
2380,ele-V9,PU200,TkEle,EGq3,GEN,pippo,<__main__.HWrapper object at 0x1169291d0>
2381,ele-V9,PU200,TkEG,EGq3,GEN,pippo,<__main__.HWrapper object at 0x11575aad0>


In [23]:
ROOT.gStyle.SetOptFit(11111)

for pu in ['PU0','PU200']:
    for tp in ['EG', 'TkEle', 'TkEG']:
        for tp_sel in ['EGq3']:
            hsets, labels, text = hplot.get_histo('pippo', sample, pu, tp, tp_sel, ['GENEtaBC'])
            hsets[0].Fit('pol1', 'S')
    
            draw(hsets, labels, text=text, min_y=0, max_y=60, y_axis_label='p_{T} @ 95% eff. [GeV] ', x_axis_label='p_{T} thresh. [GeV]')


****************************************
Minimizer is Linear
Chi2                      =      1.30501
NDf                       =            1
p0                        =      3.84476   +/-   1.745       
p1                        =      1.14534   +/-   0.0807777   

****************************************
Minimizer is Linear
Chi2                      =      2.34869
NDf                       =            1
p0                        =      1.28923   +/-   2.341       
p1                        =      1.22828   +/-   0.108367    

****************************************
Minimizer is Linear
Chi2                      =      1.44173
NDf                       =            1
p0                        =      3.93929   +/-   1.83413     
p1                        =      1.14605   +/-   0.0849037   

****************************************
Minimizer is Linear
Chi2                      =   0.00491877
NDf                       =            1
p0                        =      4.28555   +/-   0.1